In [2]:
import torch
import torch.nn.functional as F
from sae_lens import SAE
from transformer_lens import HookedTransformer
from bidict import bidict
from torchvision import datasets, transforms
import numpy as np
import time
import os
from datasets import load_dataset
import lightgbm as lgb
from sklearn.model_selection import train_test_split

def get_saes(device, total_layers=26):
    """Loads a series of Sparse Autoencoders (SAEs) for specified layers"""
    saes = []
    print(f"Loading {total_layers} SAEs...")
    for layer in range(total_layers):
        sae, _, _ = SAE.from_pretrained(
            release="gemma-scope-2b-pt-res-canonical",
            sae_id=f"layer_{layer}/width_16k/canonical",
            device=device
        )
        saes.append(sae)
        print(f"Layer {layer} loaded.")
    return saes

def get_all_mmlu_questions(dataset):
    """Returns all questions from the MMLU dataset"""
    all_questions = []
    for i in range(len(dataset)):
        data_point = dataset[i]
        question = data_point['question']
        choices = data_point['choices']
        instruction = "The following is a multiple choice question. Output only a single token corresponding to the right answer (ie: A) \n"
        formatted_question = instruction + f" Question: {question}\n"
        choice_labels = ['A', 'B', 'C', 'D']
        for j, choice in enumerate(choices):
            formatted_question += f"{choice_labels[j]}) {choice}\n"
        formatted_question += "Answer: "
        all_questions.append({
            'text': formatted_question,
            'subject': data_point['subject'],
            'answer': data_point['answer'],
            'choices': data_point['choices'],
            'raw_question': question
        })
    return all_questions
        
def process_question(model, saes, question, feature_bidict, sample_idx=None):
    """
    Performs a forward pass, extracts features and loss for a single MMLU question
    """
    question_text = question['text']
    correct_answer_idx = question['answer']
    choice_labels = ['A', 'B', 'C', 'D']
    total_features = len(feature_bidict)
    feature_vector = np.zeros(total_features, dtype=np.byte)

    with torch.no_grad():
        input_tokens = model.to_tokens(question_text, prepend_bos=True).to(model.cfg.device)
        logits, cache = model.run_with_cache(input_tokens)
        last_token_logits = logits[0, -1, :]
        choice_token_ids = [model.to_tokens(label, prepend_bos=False)[0, 0].item() for label in choice_labels]
        choice_logits = last_token_logits[choice_token_ids]
        output_logits = choice_logits.cpu().numpy() 
        loss = F.cross_entropy(choice_logits.unsqueeze(0), torch.tensor([correct_answer_idx]).to(model.cfg.device)).item()
        predicted_choice_idx = torch.argmax(choice_logits).item()
        is_correct = predicted_choice_idx == correct_answer_idx

        print(f"\n--- Sample {sample_idx} ---")
        print(f"Subject: {question['subject']}")
        print(f"Question: {question['raw_question']}")
        print("Choices:")
        for i, choice in enumerate(question['choices']):
            print(f"  {choice_labels[i]}) {choice}")
        print(f"Correct Answer: {choice_labels[correct_answer_idx]} ({question['choices'][correct_answer_idx]})")
        print(f"Model's Answer: {choice_labels[predicted_choice_idx]} ({question['choices'][predicted_choice_idx]})")
        print(f"Result: {'✓ CORRECT' if is_correct else '✗ INCORRECT'}")
        print(f"Loss: {loss:.4f}")

        for layer_idx, sae in enumerate(saes):
            final_token_activations = cache[sae.cfg.hook_name][0, -1, :].unsqueeze(0)
            feature_acts = sae.encode(final_token_activations)
            active_indices = torch.where(feature_acts > 0)[1].cpu().tolist()
            for feature_idx in active_indices:
                global_feature_idx = feature_bidict.get((layer_idx, feature_idx))
                if global_feature_idx is not None:
                    feature_vector[global_feature_idx] = 1

    return feature_vector, is_correct, output_logits
        
def extract_features_and_correctness(model, saes, questions, feature_bidict, output_dir=None):
    """
    Processes questions to get features and correctness, with an option to save data
    """
    all_feature_vectors = []
    all_correctness_labels = []
    all_output_logits = []
    for i, question in enumerate(questions):
        feature_vector, is_correct, output_logits = process_question(
            model, saes, question, feature_bidict, sample_idx=i+1
        )
        all_feature_vectors.append(feature_vector)
        all_correctness_labels.append(is_correct)
        all_output_logits.append(output_logits)

    if output_dir:
        features_np = np.array(all_feature_vectors, dtype=np.byte)
        correctness_np = np.array(all_correctness_labels)
        feature_path = os.path.join(output_dir, f"features.npy")
        correctness_path = os.path.join(output_dir, f"correctness.npy")
        np.save(feature_path, features_np)
        np.save(correctness_path, correctness_np)

    return np.array(all_feature_vectors, dtype=np.byte), np.array(all_correctness_labels), np.array(all_output_logits, dtype=np.byte)

def load_or_create_data(model, saes, feature_bidict, mmlu_dataset, output_dir):
    feature_path = os.path.join(output_dir, "features.npy")
    correctness_path = os.path.join(output_dir, "correctness.npy")
    output_logits_path = os.path.join(output_dir, "output_logits.npy")
    if os.path.exists(feature_path) and os.path.exists(correctness_path) and os.path.exists(output_logits_path):
        print(f"Found cached data. Loading from '{output_dir}'...")
        feature_vectors = np.load(feature_path)
        correctness_labels = np.load(correctness_path)
        output_logits = np.load(output_logits_path)
    else:
        print(f"No cached data found in '{output_dir}'. Generating data from scratch...")
        questions = get_all_mmlu_questions(mmlu_dataset)
        feature_vectors, correctness_labels, output_logits = extract_features_and_correctness(
            model, saes, questions, feature_bidict,
            output_dir=output_dir)
    return feature_vectors, correctness_labels, output_logits


# Init all variables and load SAEs for all layers
device = "cuda" if torch.cuda.is_available() else "cpu"
TEST_OUTPUT_DIR = "cached_test_data_correctness_losses_logits_2"
os.makedirs(TEST_OUTPUT_DIR, exist_ok=True)
model = HookedTransformer.from_pretrained("gemma-2-2b", device=device)
saes = get_saes(device, total_layers=26)
feature_bidict = bidict()
global_idx = 0
for layer_idx, sae in enumerate(saes):
    for feature_idx in range(sae.cfg.d_sae):
        feature_bidict[(layer_idx, feature_idx)] = global_idx
        global_idx += 1
total_features = len(feature_bidict)
print(f"Created mapping for {total_features} features across {len(saes)} layers")

# Build tree
MMLU_test_split = load_dataset("cais/mmlu", "all", split='test')
test_feature_vectors, test_correctness_labels, _ = load_or_create_data(model, saes, feature_bidict, MMLU_test_split, TEST_OUTPUT_DIR)
X_test, X_stop_test, y_test, y_stop_test = train_test_split(
    test_feature_vectors, 
    test_correctness_labels, 
    test_size=0.1,
    random_state=42
)
test_data = lgb.Dataset(X_test, label=y_test)
stop_test_data = lgb.Dataset(X_stop_test, label=y_stop_test, reference=test_data)
print(f"\nExtracted {len(test_feature_vectors)} samples for tree construction. Building NAP tree...")
start_time = time.time()
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    'max_depth': -1,
    'num_leaves': 2048,
    'min_data_in_leaf': 3,
    'min_sum_hessian_in_leaf': 1e-4,
    'feature_fraction': 1.0,
    'feature_fraction_bynode': 1.0,
    'bagging_fraction': 1.0,
    'learning_rate': 0.01,
    'lambda_l1': 0.05,
    'lambda_l2': 0.05,
    'num_threads': -1,
    'force_row_wise': True,
    'verbosity': 1,
    'seed': 42,
    'deterministic': True
}
model_lgb = lgb.train(
    params, 
    test_data, 
    num_boost_round=2000,
    valid_sets=[stop_test_data],
    valid_names=['stop_test_data'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=200)
    ]
)
print(f"LightGBM model built in {time.time() - start_time:.2f} seconds")

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 46.99it/s]


Loaded pretrained model gemma-2-2b into HookedTransformer
Loading 26 SAEs...
Layer 0 loaded.
Layer 1 loaded.
Layer 2 loaded.
Layer 3 loaded.
Layer 4 loaded.
Layer 5 loaded.
Layer 6 loaded.
Layer 7 loaded.
Layer 8 loaded.
Layer 9 loaded.
Layer 10 loaded.
Layer 11 loaded.
Layer 12 loaded.
Layer 13 loaded.
Layer 14 loaded.
Layer 15 loaded.
Layer 16 loaded.
Layer 17 loaded.
Layer 18 loaded.
Layer 19 loaded.
Layer 20 loaded.
Layer 21 loaded.
Layer 22 loaded.
Layer 23 loaded.
Layer 24 loaded.
Layer 25 loaded.
Created mapping for 425984 features across 26 layers
Found cached data. Loading from 'cached_test_data_correctness_losses_logits_2'...

Extracted 14042 samples for tree construction. Building NAP tree...
[LightGBM] [Info] Number of positive: 5236, number of negative: 7401
[LightGBM] [Info] Total Bins 115244
[LightGBM] [Info] Number of data points in the train set: 12637, number of used features: 57622
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.414339 -> initscore=-0.346057
[Light

In [9]:
#TODO: fix the issue of the weird output below 
VALIDATION_OUTPUT_DIR = "cached_validation_data_correctness_losses_logits"
os.makedirs(VALIDATION_OUTPUT_DIR, exist_ok=True)
MMLU_validation_split = load_dataset("cais/mmlu", "all", split='validation')
validation_feature_vectors, validation_correctness_labels, _ = load_or_create_data(model, saes, feature_bidict, MMLU_validation_split, VALIDATION_OUTPUT_DIR)
validation_predictions = model_lgb.predict(validation_feature_vectors)
confidence_threshold = 0.85
confident_mask = (validation_predictions > confidence_threshold) | (validation_predictions < (1 - confidence_threshold))
confident_validation_labels = validation_correctness_labels[confident_mask]
confident_predictions = (validation_predictions[confident_mask] > 0.5).astype(int)
confident_accuracy = np.mean(confident_predictions == confident_validation_labels)

print(f"Coverage of confident predictions: {np.mean(confident_mask):.3f} ({np.sum(confident_mask)}/{len(validation_feature_vectors)} validation samples)")
print(f"Accuracy on confident predictions: {np.mean(confident_validation_labels):.4f}")
print(f"Model's true validation accuracy: {np.sum(validation_correctness_labels) / len(validation_correctness_labels):.4f}")

Found cached data. Loading from 'cached_validation_data_correctness_losses_logits'...
Coverage of confident predictions: 0.000 (0/1531 validation samples)
Accuracy on confident predictions: nan
Model's true validation accuracy: 0.4161


In [4]:
def load_or_create_data(model, saes, feature_bidict, mmlu_dataset, output_dir):
    feature_path = os.path.join(output_dir, "features.npy")
    correctness_path = os.path.join(output_dir, "correctness.npy")
    output_logits_path = os.path.join(output_dir, "output_logits.npy")
    if os.path.exists(feature_path) and os.path.exists(correctness_path) and os.path.exists(output_logits_path):
        print(f"Found cached data. Loading from '{output_dir}'...")
        feature_vectors = np.load(feature_path)
        correctness_labels = np.load(correctness_path)
        output_logits = np.load(output_logits_path)
    else:
        print(f"No cached data found in '{output_dir}'. Generating data from scratch...")
        questions = get_all_mmlu_questions(mmlu_dataset)
        feature_vectors, correctness_labels, output_logits = extract_features_and_correctness(
            model, saes, questions, feature_bidict,
            output_dir=output_dir)
    return feature_vectors, correctness_labels, output_logits

def extract_output_logits(model, data_loader):
    model.eval()
    model.to(device)
    outputs = []

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            
            x = data.view(-1, 28 * 28)
            h1 = F.relu(model.fc1(x))
            h2 = F.relu(model.fc2(h1))
            output = model.fc3(h2)

            outputs.append(output)
    
    return torch.cat(outputs).numpy()


VALIDATION_OUTPUT_DIR = "cached_validation_data_correctness"
os.makedirs(VALIDATION_OUTPUT_DIR, exist_ok=True)
MMLU_validation_split = load_dataset("cais/mmlu", "all", split='validation')
validation_feature_vectors, validation_correctness_labels, output_logits = load_or_create_data(model, saes, feature_bidict, MMLU_validation_split, VALIDATION_OUTPUT_DIR)



confident_outputs_mask = np.array([max(output) > 10.2 for output in output_logits])
confident_outputs_validation_labels = test_correctness_labels[confident_outputs_mask]
outputs_coverage = np.mean(confident_outputs_mask)
print(f"Baseline coverage of confident NAPs: {outputs_coverage:.3f}")
print(f"Baseline Accuracy on confident predictions (how often model gets answer right): {np.sum(confident_outputs_validation_labels) / len(confident_outputs_validation_labels):.4f}")
print(f"Baseline Model's true test accuracy (how often model gets answer right): {np.sum(validation_correctness_labels) / len(validation_correctness_labels):.4f}")

No cached data found in 'cached_validation_data_correctness'. Generating data from scratch...

--- Sample 1 ---
Subject: abstract_algebra
Question: The cyclic subgroup of Z_24 generated by 18 has order
Choices:
  A) 4
  B) 8
  C) 12
  D) 6
Correct Answer: A (4)
Model's Answer: C (12)
Result: ✗ INCORRECT
Loss: 3.1603

--- Sample 2 ---
Subject: abstract_algebra
Question: Find the order of the factor group Z_6/<3>.
Choices:
  A) 2
  B) 3
  C) 6
  D) 12
Correct Answer: B (3)
Model's Answer: B (3)
Result: ✓ CORRECT
Loss: 0.8939

--- Sample 3 ---
Subject: abstract_algebra
Question: Statement 1 | A permutation that is a product of m even permutations and n odd permutations is an even permutation if and only if n is even. Statement 2 | Every group is isomorphic to a group of permutations.
Choices:
  A) True, True
  B) False, False
  C) True, False
  D) False, True
Correct Answer: A (True, True)
Model's Answer: D (False, True)
Result: ✗ INCORRECT
Loss: 3.1993

--- Sample 4 ---
Subject: abstract

KeyboardInterrupt: 